# Step 1: Set Up Kaggle API & Download Dataset

Step 1.1: Import and Upload kaggle.json

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json when prompted


Step 1.2: Configure Kaggle API and Download Dataset

In [ ]:
# Step 1.2: Setup Kaggle API and Download Dataset
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download Chest X-ray Pneumonia dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

# Unzip the dataset
!unzip chest-xray-pneumonia.zip


# Step 2: Data Exploration — IN PROGRESS

Step 2.1: Check Dataset Structure and Counts

In [ ]:
import os

base_dir = "chest_xray"

for split in ['train', 'val', 'test']:
    print(f"\n[{split.upper()}]")
    for category in os.listdir(os.path.join(base_dir, split)):
        folder = os.path.join(base_dir, split, category)
        count = len(os.listdir(folder))
        print(f"{category}: {count} images")


# Step 3: Data Preprocessing — IN PROGRESS

Step 3.1: Define Transforms for Train, Val, and Test

In [ ]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),                      # ← still good
    transforms.RandomRotation(10),                          # ✅ new
    transforms.ColorJitter(brightness=0.2, contrast=0.2),   # ✅ new
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


# For Validation & Testing
val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),   # ✅ Repeat grayscale to 3 channels
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


# Step 4: Load the Dataset with ImageFolder — IN PROGRESS

Step 4.1: Load Data Using ImageFolder

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Dataset paths
train_dir = "chest_xray/train"
val_dir = "chest_xray/val"
test_dir = "chest_xray/test"

# Create datasets
train_dataset = ImageFolder(train_dir, transform=train_transforms)
val_dataset = ImageFolder(val_dir, transform=val_test_transforms)
test_dataset = ImageFolder(test_dir, transform=val_test_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Step 5: Model Selection — IN PROGRESS

In [ ]:
from torchvision.models import resnet18
import torch.nn as nn
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet18
resnet_model = resnet18(weights="IMAGENET1K_V1")

# ✅ Unfreeze only the last block (layer4) and fully connected layer
for name, param in resnet_model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True   # allow training
    else:
        param.requires_grad = False  # keep frozen

# ✅ Modify final FC layer for 2 output classes (NORMAL & PNEUMONIA)
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, 2)

# Move model to GPU (if available)
resnet_model = resnet_model.to(device)


# Step 6: Define Loss Function and Optimizer — IN PROGRESS

Step 6.1: Define Criterion and Optimizer

In [ ]:
import torch.optim as optim

# ✅ Weighted CrossEntropyLoss to focus more on NORMAL class
weights = torch.tensor([2.0, 1.0]).to(device)  # Class 0 = NORMAL, Class 1 = PNEUMONIA
criterion = nn.CrossEntropyLoss(weight=weights)

# ✅ Optimizer: update only trainable parameters (layer4 + fc)
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, resnet_model.parameters()), lr=0.001
)

# Step 7: Training the Model — IN PROGRESS

Step 7.1: Train the Model

In [ ]:
# Set model to training mode
resnet_model.train()

num_epochs = 10  # You can increase this later
train_losses = []

for epoch in range(num_epochs):
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet_model(inputs)
        loss = criterion(outputs, labels)

        # Backward + optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("✅ Training Complete!")


# Step 8: Test the Model — IN PROGRESS

Step 8.1: Evaluate on Test Set

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

resnet_model.eval()

y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet_model(images)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())


# Compute scores
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Test Accuracy:  {acc:.4f}")
print(f"Test Precision: {prec:.4f}")
print(f"Test Recall:    {rec:.4f}")
print(f"Test F1 Score:  {f1:.4f}")


# Step 9: Confusion Matrix — IN PROGRESS

Step 9.1: Plot Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Create confusion matrix
cm = confusion_matrix(y_true, y_pred)
labels = train_dataset.classes  # ['NORMAL', 'PNEUMONIA']

# Plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix on Test Set")
plt.show()


# Step 10: Grad-CAM (Explainability)

In [ ]:
# Install torchcam
!pip install torchcam --quiet


Import and Set Up Grad-CAM

In [ ]:
from torchcam.methods import GradCAM
from torchcam.utils import overlay_mask
from torchvision.transforms.functional import to_pil_image
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms


Initialize Grad-CAM on ResNet18

In [ ]:
cam_extractor = GradCAM(resnet_model, target_layer="layer4")


Define Grad-CAM Function

In [ ]:
def generate_gradcam(model, image_path, target_class=None):
    model.eval()

    # Image preprocessing
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485], [0.229])
    ])

    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)
    input_tensor.requires_grad_()

    # Forward pass
    output = model(input_tensor)

    # Get class index (if not provided)
    class_idx = target_class if target_class is not None else output.argmax().item()

    # Extract CAM
    activation_map = cam_extractor(class_idx, output)[0].cpu()

    # Convert to image
    result = overlay_mask(
        to_pil_image(input_tensor.squeeze().cpu()),
        to_pil_image(activation_map, mode='F'),
        alpha=0.5
    )

    # Display the result
    plt.imshow(result)
    plt.title(f"Predicted: {class_idx}")
    plt.axis('off')
    plt.show()


Run Grad-CAM on a Sample Image

In [ ]:
generate_gradcam(resnet_model, "chest_xray/test/PNEUMONIA/person30_virus_69.jpeg")


1. Evaluate ResNet18 on Test Set

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=['NORMAL', 'PNEUMONIA']))

# Optional: Show confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['NORMAL', 'PNEUMONIA'], yticklabels=['NORMAL', 'PNEUMONIA'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - ResNet18')
plt.show()


Step 1: Save the Trained ResNet18 Model

In [ ]:
import torch

# Save trained ResNet18 model's weights
torch.save(resnet_model.state_dict(), "resnet_pneumonia_model.pth")
print("✅ Model saved as resnet_pneumonia_model.pth")


Step 2: Download the Trained Model to Your System

In [ ]:
from google.colab import files
files.download("resnet_pneumonia_model.pth")


# Gradio Web UI with ResNet18 + Grad-CAM

In [ ]:
import gradio as gr
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import resnet18
from torchcam.methods import GradCAM
from torchcam.utils import overlay_mask
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ResNet18 and your model weights
model = resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("resnet_pneumonia_model.pth", map_location=device))
model.to(device)
model.eval()

# GradCAM setup
cam_extractor = GradCAM(model, target_layer="layer4")

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229])
])

# Prediction function
def predict_with_cam(img):
    img = img.convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)
    input_tensor.requires_grad_()

    # Predict
    output = model(input_tensor)
    class_idx = output.argmax().item()
    class_name = ["NORMAL", "PNEUMONIA"][class_idx]

    # Generate Grad-CAM
    activation_map = cam_extractor(class_idx, output)[0].cpu()
    result = overlay_mask(
        to_pil_image(input_tensor.squeeze().cpu()),
        to_pil_image(activation_map, mode='F'),
        alpha=0.5
    )

    return result, f"Prediction: {class_name}"

# Launch Gradio UI
gr.Interface(
    fn=predict_with_cam,
    inputs=gr.Image(type="pil", label="Upload Chest X-ray"),
    outputs=[
        gr.Image(label="Grad-CAM Heatmap"),
        gr.Label(label="Diagnosis")
    ],
    title="🩺 Pneumonia Detection with ResNet18 + Grad-CAM",
    description="Upload a chest X-ray image to classify and visualize the result using ResNet18."
).launch(share=True)
